## TODO
 - Make a list of important dates (holidays, fast sundays, stake conferences)
 - Add a check for all future-scheduled dates that they don't have any hymns that were sung recently. This would happen if a hymn got changed after the schedule was made.

# Steps to use:
 1. Make sure the history is accurate for the past few weeks
 2. Check if there are any holidays, fast sundays, or special meetings coming up
 3. Run most of the notebook to generate suggestions
 4. Select hymns, enter on Edify
 5. Add selected hymns to history
 6. Add hymns to Gospel Library
 7. Update ward program, if needed, using Gospel Library as reference


In [1]:
import datetime as dt

import numpy as np
import polars as pl

pl.Config().set_tbl_rows(30)
pl.Config().set_fmt_str_lengths(50)

polars.config.Config

In [2]:
def load_history() -> pl.DataFrame:
    history_df = pl.read_csv('history.csv', infer_schema_length=None)
    history_df = history_df.with_columns(
        pl.col("date").str.strptime(pl.Date, "%m/%d/%Y").alias("date"),
    ).drop('name').drop_nulls()
    return history_df

def error_check_history() -> None:
    """Check that no future dates are duplicates with any dates since 8 weeks ago."""
    history_df = load_history()
    num_weeks_cutoff = 8
    today = dt.datetime.today().date()
    lookback_date = today - dt.timedelta(days=7 * num_weeks_cutoff)
    recent_and_future_history = history_df.filter(pl.col("date") >= lookback_date)
    value_counts = recent_and_future_history["id"].value_counts()
    recent_and_future_history = recent_and_future_history.join(
        value_counts, on="id", how="left"
    ).rename({"count": "recent_count"})

    future_history = recent_and_future_history.filter(pl.col("date") >= today)
    duplicates = future_history.filter(pl.col("recent_count") > 1)
    if len(duplicates) > 0:
        display(duplicates)
        print(">>> WARNING! DUPLICATES FOUND! <<<\n")
        raise RuntimeWarning(">>> WARNING! DUPLICATES FOUND! <<<")

def add_last_sung_col(df):
    history_df = load_history()
    last_sung = history_df.group_by('id').agg(pl.max('date').alias('last_sung'))
    df = df.join(last_sung, on='id', how='left')
    return df


def get_ranking_score(df: pl.DataFrame, type_col: str, popularity_weight=0.5, noise=0.0) -> pl.Series:
    score_df = df.filter((pl.col(type_col) == 1) & (pl.col('flagged').fill_null(0) != 1))
    weeks_since_fill_value = 100
    # Add days since last sung column
    score_df = score_df.with_columns(
        ((pl.lit(dt.datetime.today()) - pl.col('last_sung')).dt.total_days()//7)
        .fill_null(weeks_since_fill_value)
        .alias('weeks_since')
    )

    midpoint = 7
    slope = 2

    def popularity_func(x): return np.round(
        1 / (1 + np.exp(-slope * (x - midpoint))), 2)
    weeks_cutoff = 8
    score_df = score_df.with_columns((pl.col('popularity') + pl.col('popularity_adjustment').fill_null(0)).alias('adj_popularity'))
    score_df = score_df.with_columns(
        pl.col('adj_popularity').map_elements(popularity_func).alias('popularity_score'),
        pl.when(pl.col('weeks_since') > weeks_cutoff)
        .then((pl.col('weeks_since') / (weeks_since_fill_value/2)).round(3))
        .otherwise(-100).alias('weeks_since_score'),
        pl.Series(np.round(np.random.uniform(-noise, noise,
                  len(score_df)), 2)).alias('noise'),
    ).with_columns(
        (pl.col('popularity_score') * popularity_weight + pl.col('weeks_since_score')
         * (1 - popularity_weight)).alias('score') + pl.col('noise')
    )

    score_df = score_df.select([
        'id', 'name', 'length', 'adj_popularity', 'weeks_since', 'popularity_score', 'weeks_since_score', 'score'
    ]).sort('score', descending=True)

    return score_df

In [3]:
error_check_history()

In [4]:
df = pl.read_csv("hymns.csv")
df = add_last_sung_col(df)

## Sacrament Hymn

In [5]:
sacrament_ranking = get_ranking_score(df, 'is_sacrament', noise=0.1)
display(sacrament_ranking.head(10))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
197,"""O Savior, Thou Who Wearest a Crown""","""4:16""",6,99,0.12,1.98,0.99
170,"""God, Our Father, Hear Us Pray""","""2:21""",10,30,1.0,0.6,0.9
172,"""In Humility, Our Savior""","""2:46""",10,35,1.0,0.7,0.86
169,"""As Now We Take the Sacrament""","""3:01""",10,31,1.0,0.62,0.83
176,"""Tis Sweet to Sing the Matchless Love (MEREDITH)""","""2:55""",10,25,1.0,0.5,0.8
191,"""Behold the Great Redeemer Die""","""3:19""",10,21,1.0,0.42,0.78
185,"""Reverently and Meekly Now""","""5:06""",8,29,0.88,0.58,0.75
195,"""How Great the Wisdom and the Love""","""3:57""",10,22,1.0,0.44,0.74
178,"""O Lord of Hosts""","""2:12""",9,16,0.98,0.32,0.73


## General Hymns

In [6]:
general_ranking = get_ranking_score(df, 'is_general', noise=0.15, popularity_weight=0.7)
display(general_ranking.head(30))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
298,"""Home Can Be a Heaven on Earth""","""2:38""",9,29,0.98,0.58,1.01
230,"""Scatter Sunshine""","""2:48""",7,100,0.5,2.0,0.99
26,"""Joseph Smith's First Prayer""","""3:37""",8,38,0.88,0.76,0.984
223,"""Have I Done Any Good?""","""2:46""",8,37,0.88,0.74,0.968
136,"""I Know That My Redeemer Lives""","""4:19""",10,20,1.0,0.4,0.94
6,"""Redeemer of Israel""","""2:45""",9,32,0.98,0.64,0.938
89,"""The Lord Is My Light""","""3:39""",10,16,1.0,0.32,0.936
140,"""Did you Think to Pray?""","""3:04""",9,16,0.98,0.32,0.932
292,"""O My Father""","""4:25""",8,30,0.88,0.6,0.926


# Add to History

In [7]:
raise KeyboardInterrupt("This is here to stop you from running the whole notebook.")

KeyboardInterrupt: This is here to stop you from running the whole notebook.

In [8]:
def name_from_id(id, trim_parens=False):
    name_row = df.filter(pl.col("id") == id).select("name")
    if len(name_row) != 1:
        raise ValueError(f"Expected 1 row, got {len(name_row)}")
    name = name_row["name"].item()

    if trim_parens:
        if name.endswith(")") and "(" in name:
            name = name[:name.rfind("(")].strip()
    
    return name


def format_for_csv(date, opening, sacrament, intermediate, closing):
    str_list = []
    if opening:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},opening,{opening},\"{name_from_id(opening)}\""
        )
    if sacrament:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},sacrament,{sacrament},\"{name_from_id(sacrament)}\""
        )
    if intermediate:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},intermediate,{intermediate},\"{name_from_id(intermediate)}\""
        )
    if closing:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},closing,{closing},\"{name_from_id(closing)}\""
        )
    csv_str = "\n".join(str_list)
    return csv_str


def format_for_paste(opening, sacrament, intermediate, closing):
    str_list = []
    if opening:
        str_list.append(
            f"{opening} - {name_from_id(opening, trim_parens=True)}",
        )
    else:
        str_list.append("~")
    if sacrament:
        str_list.append(
            f"{sacrament} - {name_from_id(sacrament, trim_parens=True)}",
        )
    else:
        str_list.append("~")
    if intermediate:
        str_list.append(
            f"{intermediate} - {name_from_id(intermediate, trim_parens=True)}",
        )
    else:
        str_list.append("~")
    if closing:
        str_list.append(
            f"{closing} - {name_from_id(closing, trim_parens=True)}",
        )
    else:
        str_list.append("~")
    paste_str = "\n".join(str_list)
    return paste_str

date = dt.date(2025, 2, 9) # EDIT ME!
opening = 6
sacrament = 172
intermediate = 230
closing = 136

In [9]:
csv_str = format_for_csv(date, opening, sacrament, intermediate, closing)
print("Does this look right?")
print(csv_str)

Does this look right?
02/09/2025,opening,6,"Redeemer of Israel"
02/09/2025,sacrament,172,"In Humility, Our Savior"
02/09/2025,intermediate,230,"Scatter Sunshine"
02/09/2025,closing,136,"I Know That My Redeemer Lives"


In [10]:
# Append to history csv
with open('history.csv', 'a') as f:
    f.write(csv_str)
    f.write("\n")

In [11]:
# In Google Sheets, paste with *CTRL + SHIFT + V*
paste_str = format_for_paste(opening, sacrament, intermediate, closing)
print(paste_str)

6 - Redeemer of Israel
172 - In Humility, Our Savior
230 - Scatter Sunshine
136 - I Know That My Redeemer Lives
